In [1]:
import psi4
import numpy as np
from helper_CQED_RHF import *

In [2]:

# options for H2O
psi4_options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}

psi4.set_options(psi4_options_dict)
# molecule string for H2O
molecule_string = """

0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
symmetry c1
"""

# electric field for H2O - polarized along z-axis with mangitude 0.05 atomic units
lambda_vector = np.array([0.0, 0.0, 0.00])

# molecule string for H2O
mol_str_fx = """

0 1
    O      0.001   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""


# molecule string for H2O
mol_str_bx = """

0 1
    O      -0.001   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""

test_string = """
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
symmetry c1
"""
test_mol = psi4.geometry(test_string)
et, wfn_t = psi4.energy("scf", return_wfn=True)


Scratch directory: /tmp/

*** tstart() called on Jessicas-iMac.local
*** at Thu May 15 13:56:07 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass 

In [3]:
# run a psi4 rhf calculation and return the wavefunction object

# sest up molecule object
molecule_fx = psi4.geometry(mol_str_fx)
e_f, wfn_f = psi4.energy("scf",return_wfn=True)

nuc_f = molecule_fx.nuclear_dipole()


molecule_bx = psi4.geometry(mol_str_bx)
e_b, wfn_b = psi4.energy("scf", return_wfn=True)
nuc_b = molecule_bx.nuclear_dipole()




Scratch directory: /tmp/

*** tstart() called on Jessicas-iMac.local
*** at Thu May 15 13:56:08 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: Cs

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass  

  ==> Integral Setup <==

  Using in-core PK algorithm.
   Calculation information:
      Number of atoms:                   3
      Number of AO shells:              12
      Number of primitives:             32
      Number of atomic orbitals:        25
      Number of basis functions:        24

      Integral cutoff                 1.00e-12
      Number of threads:                 1

  Performing in-core PK
  Using 90300 doubles for integral storage.
  We computed 3081 shell quartets total.
  Whereas there are 3081 unique shell quartets.

  ==> DiskJK: Disk-Based J/K Matrices <==

    J tasked:                  Yes
    K tasked:                  Yes
    wK tasked:                  No
    Memory [MiB]:              375
    Schwarz Cutoff:          1E-12

    OpenMP threads:              1

  Minimum eigenvalue in the overlap matrix is 3.7382464642E-02.
  Reciprocal condition number of the overlap matrix is 1.0328870366E-02.
    Using symmetric orthogonalization.

  ==> Pre-Iteration

In [4]:
print(nuc_f)
print(nuc_b)

atom_0_mu_nuc_x_deriv = (nuc_f[0] - nuc_b[0]) / (2 * 0.001 * 1.88973)
print(atom_0_mu_nuc_x_deriv)

[ 0.0151178, 0, 1.01908 ]
[ -0.0151178, 0, 1.01908 ]
7.999983597478277


In [5]:
# run a psi4 rhf calculation and return the wavefunction object

# sest up molecule object
molecule = psi4.geometry(molecule_string)
nuc_d = molecule.nuclear_dipole()

# get the nuclear repulsion contribution to the gradient
nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())

# Set up Psi4 options
psi4.set_options(psi4_options_dict)

# run energy calculation and return wavefunction
rhf_e , wfn = psi4.energy('scf', return_wfn=True)

# get the rhf gradient
_expected_gradient = np.asarray(psi4.gradient('scf'))

# compute the QED-RHF energy and density matrix
cqed_dict = cqed_rhf(lambda_vector, molecule_string, psi4_options_dict)

# parse dictionary for ordinary RHF and CQED-RHF energy
_rhf_e = cqed_dict["RHF ENERGY"]
_cqed_rhf_e = cqed_dict["CQED-RHF ENERGY"]

# confirm the rhf energy from this method mmatches psi4
assert np.isclose(_rhf_e, rhf_e)

# parse dictionary for density matrix
_cqed_rhf_D = cqed_dict["CQED-RHF DENSITY MATRIX"]
_cqed_rhf_C = cqed_dict["CQED-RHF C"]
_cqed_rhf_eps = cqed_dict["CQED-RHF EPS"]
_cqed_rhf_F = cqed_dict["CQED-RHF FOCK"]






Scratch directory: /tmp/

*** tstart() called on Jessicas-iMac.local
*** at Thu May 15 13:56:10 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass 

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /opt/anaconda3/envs/psi4env/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
  

In [6]:
# define the molecule
molecule = psi4.geometry(molecule_string)

# get number of atoms
n_atoms = molecule.natom()

# get the nuclear gradient as a 1D numpy array
#self.nuclear_energy_gradient =  np.asarray(molecule.nuclear_repulsion_energy_deriv1()).flatten()#

# Get the RHF wavefunction
#wfn = self.compute_qed_rhf_wfn()

# get the number of orbitals and the number of doubly occupied orbitals
n_orbitals = wfn.nmo()
n_docc = wfn.nalpha()

# get the orbital transformation matrix
C = wfn.Ca() # -> as psi4 matrix object
Cnp = np.asarray(C) # -> as numpy array

# get the Density matrix by summing over the occupied orbital transformation matrix
Cocc = Cnp[:, :n_docc]
density_matrix = np.einsum("pi,qi->pq", Cocc, Cocc)

# get Da and Db from wfn object -> might be redundant
Da = np.asarray(wfn.Da())
Db = np.asarray(wfn.Db())

# D symmetrized
D_sym = 0.5 * (Da + Db) + 0.5 * np.einsum('rs -> sr', (Da + Db))

assert np.allclose(D_sym, 2 * density_matrix, 1e-11, 1e-11)

D = psi4.core.Matrix.from_array(2 * density_matrix)

# origin vector
origin = [0.0, 0.0, 0.0]

# instantiate the MintsHelper object
mints = psi4.core.MintsHelper(wfn.basisset())

# get the derivatives of the dipole integrals
#mu_mo = np.asarray(mints.dipole_grad(D))[0]
#print("Shape of mu_mo: ", mu_mo.shape)

# get the derivatives of the quadrupole integrals
max_order = 1
q_mo = np.asarray(mints.multipole_grad(D, max_order, origin))
#print(q_mo[:,0])
q_x = q_mo[:,2].reshape(3,3)
print(q_x)

AttributeError: 'psi4.core.MintsHelper' object has no attribute 'multipole_grad'

In [16]:
# initialize array for the integral derivatives

mu_deriv_matrix_ao = np.asarray(mints.ao_elec_dip_deriv1(0))
print(np.shape(mu_deriv_matrix_ao))

(9, 24, 24)


In [ ]:
print(_expected_gradient - np.asarray(scf_gradient))
#print(np.asarray(electronic_gradient))
#print(nuclear_repulsion_gradient)
#print(nuclear_repulsion_gradient + np.asarray(electronic_gradient))

In [ ]:
# update the wfn object with the coefficients and density matrix from the cavity calculation
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# now update the quantities with cqed quantities
#wfn_dict['matrix']['Ca'] = np.copy(_cqed_rhf_C)
#wfn_dict['matrix']['Cb'] = np.copy(_cqed_rhf_C)
#wfn_dict['matrix']['Da'] = np.copy(_cqed_rhf_D)
#wfn_dict['matrix']['Db'] = np.copy(_cqed_rhf_D)
#wfn_dict['matrix']['Fa'] = np.copy(_cqed_rhf_F)
#wfn_dict['matrix']['Fb'] = np.copy(_cqed_rhf_F)
#wfn_dict['vector']['epsilon_a'] = np.copy(_cqed_rhf_eps)
#wfn_dict['vector']['epsilon_b'] = np.copy(_cqed_rhf_eps)

# push these back to the wavefunction object
up_wfn = psi4.core.Wavefunction.from_file(wfn_dict)


In [ ]:
# this is the total rhf gradient with the RHF density, etc, stored in wfn
up_scf_gradient = psi4.core.scfgrad(up_wfn)

In [ ]:
#### call scfgrad for electron-only part of gradient ####
electronic_gradient = psi4.core.scfgrad()

In [ ]:
print(gradient)